# Database setup

The cell below creates the database and all associated tables. Existing tables will not be cleared. You should-re run this cell if there are changes to the database tables that should be applied.

In [1]:
%load_ext autoreload
%autoreload 2

import pickle
import logging

from itertools import chain
from typing import List, Set, Generator, Dict

from src import settings
from src.database import models
from src.constants import Paths


logger = logging.getLogger(settings.LOGGER_NAME)

# Bind models and create tables
settings.DATABASE.bind(models.MODELS, bind_refs=True, bind_backrefs=True)
settings.DATABASE.create_tables(models.MODELS)

# REST client setup

Clients for Uniprot and Kegg are also created so downloads can be cached during this session and re-used if something goes wrong.

In [2]:
from src.clients import Kegg, UniprotClient

# Use the download caches for UniProt and Kegg clients.
cache_kegg = True
cache_uniprot = True
kegg_organism = 'hsa'

# Use global clients to cache results in case of error.
kegg_client = Kegg(use_cache=cache_kegg)
uniprot_client = UniprotClient(use_cache=cache_uniprot, verbose=True)

# Annotations

The cell directly below will populate the annotation tables, which includ Pfam, InterPro and Gene Ontology terms. This is optional but if performed will result in less time being required to populate the protein and interaction tables.

In [3]:
from src.parsers import go, interpro, pfam
from src.database.utilities import create_terms
from src.database.models import GeneOntologyTerm, PfamTerm, InterproTerm


with settings.DATABASE.atomic():
    create_terms(
        terms=pfam.parse_clans_file(path=Paths.pfam_clans), 
        model=PfamTerm
    )
    create_terms(
        terms=go.parse_go_obo(path=Paths.go_obo), 
        model=GeneOntologyTerm
    )
    create_terms(
        terms=interpro.parse_entry_list(path=Paths.interpro_entries), 
        model=InterproTerm
    )

[INFO] 2019-09-08 11:21:26,024 go Unzipping '/home/daniel/.tamago/data/go.obo.gz' into '/tmp/uzippedjpddpzfm.obo'


/tmp/uzippedjpddpzfm.obo: fmt(1.2) rel(2019-06-01) 47,444 GO Terms


# Proteins and Interactions

The cells below will:

1. Parse all interaction data sources.

2. Collect a list of UniProt accessions to download XML records for.

3. Download a mapping table for the the accessions collected in step **(2)** which maps them to their most recent **primary** accessions on UniProt, or nothing if the accession is no longer valid.

4. Update the accessions of all `source` and `target` nodes to the most recent UniProt accessions. Interactions which cannot be mapped will be discarded.

5. Normalize interactions such that all fields will have the same format and can be combined seamlessly.

6. Aggregate all interactions by their `source` and `target` nodes. Metadata fields (labels, databases etc) will be combined into a single data instance.

In [4]:
from src.parsers import kegg, hprd, bioplex, pina, innate
from src.parsers.types import InteractionData
from src.clients.uniprot import UniprotEntry
from src.clients.kegg import KeggPathway
from src.database.utilities import update_accessions


# Download required pathway and ID mapping information from KEGG.
pathways: List[KeggPathway] = kegg_client.parse_all_pathways(
    organism=kegg_organism,
    verbose=True
)

logger.info(f"Downloading '{kegg_organism}' gene to UniProt conversion table.")
kegg_gene_to_uniprot: Dict[str, List[str]]
kegg_gene_to_uniprot = kegg_client.convert(source=kegg_organism)


# Normalizing interactions formats all identifiers/fields and set removes 
# duplicates.
logger.info("Parsing interactions.")
interactions: Set[InteractionData] = set(
    interaction.normalize()
    for interaction in chain(
        kegg.parse_interactions(
            pathways=pathways,
            kegg_gene_to_uniprot=kegg_gene_to_uniprot,
            exclude_labels=(
                "indirect effect",
                "compound",
                "hidden compound",
                "state change",
                "missing interaction",
            )
        ),
        hprd.parse_interactions(
            ptms=hprd.parse_ptm(path=Paths.hprd_ptms), 
            xrefs=hprd.parse_xref_mapping(path=Paths.hprd_xref)
        ),
        pina.parse_interactions(path=Paths.pina2_mitab),
        innate.parse_interactions(Paths.innate_all),
        innate.parse_interactions(Paths.innate_curated),
        bioplex.parse_interactions(path=Paths.bioplex),
    )
)

[INFO] 2019-09-08 11:21:48,819 kegg Downloading and parsing 335 'hsa' pathways.
100%|██████████| 335/335 [00:05<00:00, 63.12it/s] 
[INFO] 2019-09-08 11:21:54,138 <ipython-input-4-0061b4f498ec> Downloading 'hsa' gene to UniProt conversion table.
[INFO] 2019-09-08 11:21:54,139 <ipython-input-4-0061b4f498ec> Parsing interactions.
[INFO] 2019-09-08 11:21:59,147 kegg Parsing pathway XML files for interactions.
[WARNING] 2019-09-08 11:22:45,178 __init__ Edge ('Q6ZMN8', 'Q659C4-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,179 __init__ Edge ('Q8N8A2-2', 'O15084-1') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,180 __init__ Edge ('Q8N8A2-2', 'P46531') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,181 __init__ Edge ('Q8N8A2-2', 'Q04721') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,182 __init__ Edge ('Q8N8A2-2', 'Q9UM47') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,183 __init__ E

[WARNING] 2019-09-08 11:22:45,244 __init__ Edge ('Q99848', 'P07951-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,245 __init__ Edge ('P05111', 'Q9Y6K9-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,246 __init__ Edge ('Q96A73-2', 'P38646') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,246 __init__ Edge ('Q96A73-2', 'Q86X55-1') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,247 __init__ Edge ('O60663-2', 'Q13394') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,248 __init__ Edge ('O60663-2', 'A5A3E0') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,248 __init__ Edge ('O60663-2', 'P60709') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,249 __init__ Edge ('O60663-2', 'Q96KN3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,250 __init__ Edge ('O60663-2', 'Q7L3T8') contains UniProt isoform identifiers.
[WARNING] 2019-09

[WARNING] 2019-09-08 11:22:45,301 __init__ Edge ('Q9Y5U9', 'Q709C8-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,302 __init__ Edge ('Q9Y5U9', 'P23470-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,302 __init__ Edge ('Q8N6M8-2', 'Q96SY0') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,305 __init__ Edge ('Q8N6M8-2', 'Q9UK61-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,305 __init__ Edge ('Q8N6M8-2', 'Q96IC2-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,306 __init__ Edge ('Q8N6M8-2', 'O60343-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,307 __init__ Edge ('Q8N6M8-2', 'Q8N122') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,308 __init__ Edge ('Q8N6M8-2', 'Q9H223') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,309 __init__ Edge ('Q8N6M8-2', 'Q6AI08') contains UniProt isoform identifiers.
[WARNING] 201

[WARNING] 2019-09-08 11:22:45,379 __init__ Edge ('Q8N6N7', 'P12931-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,380 __init__ Edge ('Q8N6N7', 'P23470-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,381 __init__ Edge ('Q8N6N7', 'Q15067-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,381 __init__ Edge ('Q8N6N7', 'Q9ULP9-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,382 __init__ Edge ('Q9NRR3', 'Q99576-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,382 __init__ Edge ('Q9BUK0-2', 'P06703') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,383 __init__ Edge ('Q7Z4G1-2', 'Q86X83') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,384 __init__ Edge ('Q7Z4G1-2', 'Q9UBQ0-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,386 __init__ Edge ('Q7Z4G1-2', 'Q7Z3J2') contains UniProt isoform identifiers.
[WARNING] 2019-09

[WARNING] 2019-09-08 11:22:45,430 __init__ Edge ('Q9Y4Y9-2', 'Q96S86') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,431 __init__ Edge ('Q9Y4Y9-2', 'P62310') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,431 __init__ Edge ('Q9Y4Y9-2', 'Q8NB15-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,432 __init__ Edge ('Q9Y4Y9-2', 'P62306') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,432 __init__ Edge ('Q96F83', 'Q8TDW7-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,433 __init__ Edge ('Q9NQG1', 'Q9HD26-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,433 __init__ Edge ('Q9BQ48', 'O00410-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,434 __init__ Edge ('Q9BQ48', 'Q9NYK5-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:22:45,435 __init__ Edge ('Q9BQ48', 'Q9HD33-2') contains UniProt isoform identifiers.
[WARNING] 2019-09

...

[WARNING] 2019-09-08 11:23:04,388 __init__ Edge ('Q96Q40-3', 'Q9UN36-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,389 __init__ Edge ('Q96Q40-3', 'Q86WN1') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,390 __init__ Edge ('Q96Q40-3', 'Q01581') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,391 __init__ Edge ('Q96Q40-3', 'Q5TEC6') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,392 __init__ Edge ('Q96Q40-3', 'Q14116') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,392 __init__ Edge ('Q96Q40-3', 'Q92597') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,393 __init__ Edge ('Q96Q40-3', 'Q9UL46') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,394 __init__ Edge ('Q96Q40-3', 'O15231-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,395 __init__ Edge ('Q96Q40-3', 'P02763') contains UniProt isoform identifiers.
[WARNING] 2019-

[WARNING] 2019-09-08 11:23:04,446 __init__ Edge ('Q9UQB9-2', 'Q00535') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,447 __init__ Edge ('Q9UQB9-2', 'Q9NQS7-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,447 __init__ Edge ('Q9UQB9-2', 'Q96C57') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,448 __init__ Edge ('Q9UQB9-2', 'Q92620') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,448 __init__ Edge ('Q8N128', 'Q5SWX8-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,449 __init__ Edge ('Q8N128', 'Q658P3-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,449 __init__ Edge ('Q8N128', 'Q9NSI2-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,450 __init__ Edge ('Q8N128', 'Q7Z7G8-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,450 __init__ Edge ('Q8N128', 'Q6PIU2-2') contains UniProt isoform identifiers.
[WARNING] 2019-09

[WARNING] 2019-09-08 11:23:04,497 __init__ Edge ('P13727', 'Q9NZV5-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,497 __init__ Edge ('P13727', 'A7E2V4-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,498 __init__ Edge ('P13727', 'Q9H9V9-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,498 __init__ Edge ('P13727', 'Q86UQ0-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,499 __init__ Edge ('P13727', 'O95450-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,499 __init__ Edge ('O43683', 'O60566-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,499 __init__ Edge ('O43683', 'Q8NG31-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,500 __init__ Edge ('O43683', 'Q12830-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,500 __init__ Edge ('Q14206-2', 'P16298-2') contains UniProt isoform identifiers.
[WARNING] 2019-09

[WARNING] 2019-09-08 11:23:04,550 __init__ Edge ('Q96DC7-2', 'Q96FC7-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,551 __init__ Edge ('O15165-2', 'Q9P2P5') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,551 __init__ Edge ('O15165-2', 'Q76N89') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,552 __init__ Edge ('O15165-2', 'O00308') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,552 __init__ Edge ('O15165-2', 'P46934-4') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,553 __init__ Edge ('O15165-2', 'Q96J02') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,554 __init__ Edge ('O15165-2', 'Q96PU5-7') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,554 __init__ Edge ('O15165-2', 'Q9H0M0-3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,555 __init__ Edge ('O15165-2', 'Q86VP1-2') contains UniProt isoform identifiers.
[WARNING]

[WARNING] 2019-09-08 11:23:04,601 __init__ Edge ('Q14004-2', 'Q9H4M3-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,602 __init__ Edge ('Q14004-2', 'Q9ULE6') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,602 __init__ Edge ('Q00526', 'P07900-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,603 __init__ Edge ('Q00526', 'P49918-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,604 __init__ Edge ('P50750-2', 'Q4G0J3') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,605 __init__ Edge ('P50750-2', 'P07900-2') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,605 __init__ Edge ('P50750-2', 'P08238') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,606 __init__ Edge ('P50750-2', 'Q58FF6') contains UniProt isoform identifiers.
[WARNING] 2019-09-08 11:23:04,606 __init__ Edge ('P50750-2', 'Q58FF7') contains UniProt isoform identifiers.
[WARNING] 2019-

In [5]:
# Map all UniProt identifiers to their most recent versions.
logger.info("Fetching updated UniProt identifiers.")
uniprot_ids: Set[str] = set(
    accession
    for interaction in interactions
    for accession in (interaction.source, interaction.target)
)

uniprot_id_mapping: Dict[str, List[str]]
uniprot_id_mapping = uniprot_client.get_mapping_table(
    identifiers=uniprot_ids,
    fr='ACC+ID',
    to="ACC",
    batch_size=500,
)

# Loop through and check isoforms with no mapping. These identifiers may
# still exist but the isoform identifier has since been deleted/changed and
# UniProt is unable to map these identifiers to their primary versions.
# Clip the isoform key and link to the non-isoform version. We want to keep 
# the interactions with these isoforms as the related XML record might still 
# exist.
for key, value in uniprot_id_mapping.items():
    # Check if isoform.
    if len(key.split('-')) == 2 and not len(value):
        uniprot_id_mapping[key] = [key.split('-')[0]]

[INFO] 2019-09-08 11:23:37,212 <ipython-input-5-412b911e4ebc> Fetching updated UniProt identifiers.
[INFO] 2019-09-08 11:23:37,664 uniprot Requesting 48 batches.
100%|██████████| 48/48 [06:15<00:00,  7.82s/it]


In [6]:
# Map all source and target nodes to their most recent accessions. This 
# call will also aggregate all equal interactions into a single instance
# after the mapping process.
logger.info("Mapping interactions to updated UniProt identifiers.")
mapped_interactions: List[InteractionData]
mapped_interactions = update_accessions(
    interactions=interactions,
    mapping=uniprot_id_mapping,
    keep_isoforms=True,
)

logger.info("Aggregating interactions by souce and target nodes.")
aggregated_interactions: List[InteractionData]
aggregated_interactions = InteractionData.aggregate(i.normalize() for i in mapped_interactions)

[INFO] 2019-09-08 11:29:53,070 <ipython-input-6-141f2b885137> Mapping interactions to updated UniProt identifiers.
[WARNING] 2019-09-08 11:29:53,185 utilities Could not find a mapping for target node 'P53TG1'. Skipping interaction {'source': 'P05067', 'target': 'P53TG1', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': '21832049', 'psimi': 'MI:0096'}]}.
[WARNING] 2019-09-08 11:29:53,296 utilities Could not find a mapping for target node 'P170P1'. Skipping interaction {'source': 'Q9BU64', 'target': 'P170P1', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': '16189514', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-08 11:29:53,297 utilities Could not find a mapping for target node 'P170P1'. Skipping interaction {'source': 'Q9BU64', 'target': 'P170P1', 'labels': [], 'databases': ['innatedb'], 'evidence': [{'pubmed': '19447967', 'psimi': 'MI:0018'}]}.
[WARNING] 2019-09-08 11:29:53,303 utilities Could not find a mapping for target node 'P170P1'. Skipping interactio

# Querying UniProt

The cell below queries UniProt for the XML flat files for each uniprot identifier. This can take a while if this is a first time run, or there are a lot of cache misses. Try again with a lower `batch_size` if you see a `Connection reset` or similar error.

In [11]:
uniprot_entries: Generator[UniprotEntry, None, None]
uniprot_entries = (
    (identifier, entry) for (identifier, entry)
    in uniprot_client.get_entries(
        identifiers=uniprot_ids, 
        batch_size=500
    )
    if entry is not None
)

# Database population

The cell below will parse and validate all interactions, including UniProt identifiers and associated metadata, and then commit these to the database. Additional metadata will be associated with interactions such as PubMed/PSIMI identifiers, labels and the source databases. Directionality information is retained such that separate rows will be added for the interactions `(A, B)` and `(B, A)` where `A` is the source node and `B` is the target node. This information is particularly useful for analysing Kegg interactions, which have directionality information in the labels.

In [12]:
from src.database.utilities import create_proteins


proteins: List[models.Protein]
proteins = create_proteins(uniprot_entries)
logger.info(f"Created {len(proteins)} protein rows")

[INFO] 2019-09-08 13:12:38,757 uniprot Requesting 1 batches.
100%|██████████| 1/1 [00:06<00:00,  6.92s/it]
[INFO] 2019-09-08 13:12:45,679 uniprot Parsing XML into UniprotEntry instances.
[WARNING] 2019-09-08 13:13:02,498 uniprot Could not find any record for 'A2A284'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:02,947 uniprot Could not find any record for 'A2A457'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:02,948 uniprot Could not find any record for 'A2A6V3'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:07,822 uniprot Could not find any record for 'A6NEJ5'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:07,884 uniprot Could not find any record for 'A6NF12'. If it is a valid UniProt identifier it may be obsolete or t

[WARNING] 2019-09-08 13:13:38,325 uniprot Could not find any record for 'E9PHH6'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:38,944 uniprot Could not find any record for 'F4EBP3'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:38,996 uniprot Could not find any record for 'F5GZ33'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:38,997 uniprot Could not find any record for 'F5GZ62'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:38,998 uniprot Could not find any record for 'F5GZM1'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:13:38,999 uniprot Could not find any record for 'F5H0Y5'. If it is a valid UniProt identifier it may be obsolete or the entry no longer 

[WARNING] 2019-09-08 13:22:46,963 uniprot Could not find any record for 'Q5T7C6'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:22:51,806 uniprot Could not find any record for 'Q5T8Z0'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:22:51,975 uniprot Could not find any record for 'Q5T9K6'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:22:52,060 uniprot Could not find any record for 'Q5T9M4'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:22:52,921 uniprot Could not find any record for 'Q5TBK5'. If it is a valid UniProt identifier it may be obsolete or the entry no longer exists.
[WARNING] 2019-09-08 13:22:52,922 uniprot Could not find any record for 'Q5TBT1'. If it is a valid UniProt identifier it may be obsolete or the entry no longer 

{'go_terms': {GeneOntologyTermData(identifier='GO:0043025', category='Cellular component', obsolete=False, name='neuronal cell body', description=None), GeneOntologyTermData(identifier='GO:0035145', category='Cellular component', obsolete=False, name='exon-exon junction complex', description=None), GeneOntologyTermData(identifier='GO:0008380', category='Biological process', obsolete=False, name='RNA splicing', description=None)}, 'interpro_terms': {InterproTermData(identifier='IPR004023', name='Mago_nashi', description=None, entry_type=None), InterproTermData(identifier='IPR036605', name='Mago_nashi_sf', description=None, entry_type=None)}, 'pfam_terms': {PfamTermData(identifier='PF02792', name='Mago_nashi', description=None)}, 'keywords': set(), 'genes': {GeneData(symbol='HCG_1773848', relation='orf'), GeneData(symbol='FLJ10292', relation='primary')}, 'accessions': {'A0A023T6R1'}, 'taxonomy': 9606, 'reviewed': False, 'version': '44', 'sequence': 'MAVASDFYLRYYVGHKGKFGHEFLEFEFRPDGKLRYAN

IntegrityError: NOT NULL constraint failed: proteindata.created

In [ ]:
from src.database.utilities import create_interactions


interactions: List[models.Interaction]
interactions = create_interactions(mapped_interactions)
logger.info(f"Created {len(interactions)} interaction rows")